In [5]:
# Import necessary libraries


import pandas as pd 
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
import os, types
from botocore.client import Config
import ibm_boto3
from ibm_watson_machine_learning import APIClient
import ast
import json

In [6]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='HfjmkDQ5vWAr_zPsfU9uTM7wBE5LfIzwTjG5abrXDsaT',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu-de.cloud-object-storage.appdomain.cloud')

bucket = 'mediamonitoring-donotdelete-pr-il1ydrrkdjxjxm'
object_key = 'BBC News Train.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data = pd.read_csv(body)
data.head()



,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens in ...,business
3,1976,lifestyle governs mobile choice faster bett...,tech
4,917,enron bosses in $168m payout eighteen former e...,business


In [7]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['Text'], data['Category'], test_size=0.2, random_state=42)

# Text preprocessing using TF-IDF vectorization
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)



In [8]:
# Initialize and train the SVC classifier
svc_classifier = SVC(kernel='linear', C=1.0)
svc_classifier.fit(X_train_tfidf, y_train)

# Predict labels on the test set
y_pred = svc_classifier.predict(X_test_tfidf)

# Evaluate the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

     business       0.95      0.99      0.97        75
entertainment       0.98      0.98      0.98        46
     politics       0.98      0.98      0.98        56
        sport       1.00      1.00      1.00        63
         tech       1.00      0.95      0.97        58

     accuracy                           0.98       298
    macro avg       0.98      0.98      0.98       298
 weighted avg       0.98      0.98      0.98       298



In [9]:
# Initialize and train the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_tfidf, y_train)

# Predict labels on the test set
y_pred = rf_classifier.predict(X_test_tfidf)

# Evaluate the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

     business       0.90      0.97      0.94        75
entertainment       1.00      0.91      0.95        46
     politics       0.96      0.96      0.96        56
        sport       1.00      1.00      1.00        63
         tech       1.00      0.97      0.98        58

     accuracy                           0.97       298
    macro avg       0.97      0.96      0.97       298
 weighted avg       0.97      0.97      0.97       298



In [10]:
# Initialize and train the Multinomial Naive Bayes Classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf, y_train)

# Predict labels on the test set for Multinomial Naive Bayes Classifier
y_pred_nb = nb_classifier.predict(X_test_tfidf)


# Evaluate the Multinomial Naive Bayes Classifier
print("Multinomial Naive Bayes Classifier Classification Report:")
print(classification_report(y_test, y_pred_nb))

Multinomial Naive Bayes Classifier Classification Report:
               precision    recall  f1-score   support

     business       0.95      0.97      0.96        75
entertainment       1.00      0.93      0.97        46
     politics       0.92      0.96      0.94        56
        sport       0.94      1.00      0.97        63
         tech       0.98      0.88      0.93        58

     accuracy                           0.95       298
    macro avg       0.96      0.95      0.95       298
 weighted avg       0.95      0.95      0.95       298



In [11]:
# Save the trained model to a file (e.g., using joblib or pickle)
import joblib
joblib.dump(svc_classifier, 'svc_model.pkl')

['svc_model.pkl']

In [12]:
import pandas as pd 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
import os, types
from botocore.client import Config
import ibm_boto3
from ibm_watson_machine_learning import APIClient
import ast
import json


In [13]:

df=data
data.columns

Index(['ArticleId', 'Text', 'Category'], dtype='object')

In [14]:
# preprocessing

# df["Category"] = df["Category"].apply(lambda x: ast.literal_eval(x))
y = df["Category"]

X = df["Text"]

In [15]:
# models

mlb = MultiLabelBinarizer()
multilabel_y = mlb.fit_transform(y)
svc = LinearSVC()
clf = OneVsRestClassifier(svc)

In [16]:
pipeline = Pipeline([('vect', CountVectorizer(ngram_range=(1, 3))), 
                    ('tfidf', TfidfTransformer()), 
                    ('clf', OneVsRestClassifier(clf))])

In [20]:
# Use multilabel classes to fit the pipeline.
pipeline.fit(X, multilabel_y)

Pipeline(steps=[('vect', CountVectorizer(ngram_range=(1, 3))),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 OneVsRestClassifier(estimator=OneVsRestClassifier(estimator=LinearSVC())))])

In [21]:
# If X is a Pandas Series
# X = X.to_numpy()

AttributeError: 'numpy.ndarray' object has no attribute 'to_numpy'

In [22]:
type(X)

numpy.ndarray

In [25]:
wml_credentials = {
    "apikey": '<api_key>',
    "url": 'https://eu-de.ml.cloud.ibm.com',}


In [26]:
client = APIClient(wml_credentials)

In [27]:

client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  ---------------------------------  ------------------------
ID                                    NAME                               CREATED
e924028a-bde6-4f8c-acf2-cdb0db851228  Media Monitoring Deployment Space  2023-08-31T09:43:40.205Z
------------------------------------  ---------------------------------  ------------------------


,ID,NAME,CREATED
0,e924028a-bde6-4f8c-acf2-cdb0db851228,Media Monitoring Deployment Space,2023-08-31T09:43:40.205Z


In [28]:
space_id = 'e924028a-bde6-4f8c-acf2-cdb0db851228'

In [29]:
client.set.default_space(space_id)

'SUCCESS'

In [30]:
MODEL_NAME = 'MultiLabelModel'
DEPLOYMNET_NAME = 'LinearSVC_Deploy'
BEST_MODEL = clf

In [31]:
sofware_spec_uid = client.software_specifications.get_id_by_name("runtime-22.2-py3.10")


In [32]:
metadata = {client.repository.ModelMetaNames.NAME: 'MultiLabelModel',
            client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.1',
            client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid}

published_model = client.repository.store_model(
    model=pipeline,
    meta_props=metadata,
    training_data=X,
    training_target=multilabel_y)

In [33]:
type(multilabel_y)

numpy.ndarray

In [34]:
published_model_uid = client.repository.get_model_id(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

{
  "entity": {
    "hybrid_pipeline_software_specs": [],
    "label_column": "l0",
    "schemas": {
      "input": [
        {
          "fields": [
            {
              "name": "f1",
              "type": "str"
            }
          ],
          "id": "1",
          "type": "struct"
        }
      ],
      "output": []
    },
    "software_spec": {
      "id": "b56101f1-309d-549b-a849-eaa63f77b2fb",
      "name": "runtime-22.2-py3.10"
    },
    "training_data_references": [
      {
        "id": "1",
        "location": {},
        "schema": {
          "fields": [
            {
              "name": "f1",
              "type": "str"
            }
          ],
          "id": "1",
          "type": "ndarray"
        },
        "type": "container"
      }
    ],
    "type": "scikit-learn_1.1"
  },
  "metadata": {
    "created_at": "2023-09-04T07:07:40.418Z",
    "id": "466e7b0a-7e2b-4ac1-a54b-d727aac48ced",
    "modified_at": "2023-09-04T07:07:55.684Z",
    "name": "MultiLa

In [35]:
models_details = client.repository.list_models()

------------------------------------  ---------------  ------------------------  ----------------  ----------  ----------------
ID                                    NAME             CREATED                   TYPE              SPEC_STATE  SPEC_REPLACEMENT
466e7b0a-7e2b-4ac1-a54b-d727aac48ced  MultiLabelModel  2023-09-04T07:07:40.002Z  scikit-learn_1.1  supported
7a1a333e-bee9-4baa-b333-d702256faad8  MultiLabelModel  2023-09-03T18:14:22.002Z  scikit-learn_1.1  supported
------------------------------------  ---------------  ------------------------  ----------------  ----------  ----------------


In [36]:
metadata = {client.deployments.ConfigurationMetaNames.NAME: "Deployment of scikit model",
            client.deployments.ConfigurationMetaNames.ONLINE: {}}

created_deployment = client.deployments.create(published_model_uid, meta_props=metadata)



#######################################################################################

Synchronous deployment creation for uid: '466e7b0a-7e2b-4ac1-a54b-d727aac48ced' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
.
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='86f7ca55-d133-41be-b62d-7ffa03624818'
------------------------------------------------------------------------------------------------


